<a href="https://colab.research.google.com/github/jcandane/RCF/blob/main/gpy_play.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import scipy

try:
    import GPy
except:
    !pip install gpy==1.13.1
    import GPy

import plotly.graph_objects as go

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 5.0 MB/s eta 0:00:00


also check out: https://www.pymc.io/projects/examples/en/latest/gaussian_processes/GP-Latent.html

In [2]:
d = 1 # input dimension
var = 1. # variance
theta = 0.2 # lengthscale
k = GPy.kern.RBF(d)

In [3]:
kk = GPy.kern.RBF(d)

kk.parameter_names()

kk.variance = 3.5

k.param_array

array([1., 1.])

In [4]:
yo = {
"lengthscale":8.1,
 "variance":2.5
}

GPy.kern.RBF(2, **yo )

rbf.,value,constraints,priors
variance,2.5,+ve,
lengthscale,8.1,+ve,


In [5]:
type( k.parameters[0] )

GPy.core.parameterization.param.Param

In [6]:
k.__dict__

{'_name': 'rbf',
 'observers': [(-100, <weakref at 0x7fdb88cb0900; to 'RBF' at 0x7fdb7bebd510>, <bound method Parameterizable._parameters_changed_notification of <GPy.kern.src.rbf.RBF object at 0x7fdb7bebd510>>)],
 '_update_on': True,
 '_index_operations': OrderedDict([('constraints',
               <paramz.core.index_operations.ParameterIndexOperations at 0x7fdb5070cfa0>),
              ('priors',
               <paramz.core.index_operations.ParameterIndexOperations at 0x7fdb5070ceb0>)]),
 '_default_constraint_': None,
 '_optimizer_copy_': None,
 '_optimizer_copy_transformed': False,
 'parameters': [rbf.variance:
  Param([1.]),
  rbf.lengthscale:
  Param([1.])],
 '_param_array_': array([1., 1.]),
 '_added_names_': set(),
 'logger': <Logger RBF (WARNING)>,
 '_Parameterizable__visited': False,
 'cache': {},
 '_default_prior_': None,
 'size': 2,
 '_fixes_': None,
 '_param_slices_': [slice(0, 1, None), slice(1, 2, None)],
 'input_dim': 1,
 'active_dims': array([0]),
 '_all_dims_active': a

In [7]:
print( k.parameters ) ## define kernel with these parameters????

#kk = GPy.kern.RBF(1, k.parameters)

#GPy.kern.RBF(**k.__dict__)

[rbf.variance:
Param([1.]), rbf.lengthscale:
Param([1.])]


In [8]:
R_ix = np.random.rand(5,1)
R_jx = np.random.rand(8,1)

k.K(R_ix, R_jx).shape

L_ij = np.linalg.cholesky( k.K(R_ix) )

scipy.linalg.cho_solve((L_ij, True), R_ix)

array([[ -32.429016  ],
       [ 321.50558684],
       [-497.73653793],
       [   1.25170152],
       [ 208.56321486]])

In [9]:
Domain = np.array([[0,10.],[-3,4.]], dtype=np.float64) #torch.tensor([[0,10.],[-3,4.],[-8,-2]]) ### numpy.2darray
N      = 32  ### number of defining points
MO     = 1   ### int (dimension of OUT)

kernel = GPy.kern.RBF(Domain.shape[0])
seed   = 1372
############################

np.random.seed( 777 )
R_ix  = np.random.rand( N, Domain.shape[0] )
R_ix  = (Domain[:,1] - Domain[:,0]) * R_ix
R_ix += Domain[:,0] ## save this!!!!!

L_ij = np.linalg.cholesky( kernel.K(R_ix) )

D_iX  = np.random.normal(0,1,size=(N, MO))
D_iX *= np.diag(L_ij)[:,None]
D_iX  = L_ij @ D_iX

S_jX  = scipy.linalg.cho_solve((L_ij, True), D_iX) ## save this!!!!!

### the RCF class for numpy/GPy

In [14]:
import numpy as np
import GPy

class RCF():
    """ built: 3/19/2024
    this an object of a Random-Contionus-Function (RCF), with-respect-to a gpy kernel
    RCF : IN -> OUT = R^(MO)
    we define a prior, and then sample to form a posterior.
    """

    def __init__(self, Domain:np.ndarray, N:int, MO:int=1, seed:int=777,
                 IN_noise=None, OUT_noise=None,
                 kernel=GPy.kern.RBF):
        """ !! note datatypes should be tf.float64 for stable Cholesky-operations
        GIVEN >
             Domain : 2d-np.ndarray (with shape=(d,2), with d=# of dims )
                  N : int (number-of-defining-points)
                 MO : int (Multiple-Output Dimension)
             **seed : int
           **kernel : GPy.kern
         **IN_noise : 1d-np.ndarray (len == Domain.shape[1])
        **OUT_noise : 1d-np.ndarray (len == MO)

        GET   >
            None
        """

        self.dtype  = np.float64
        self.IN     = Domain.astype(self.dtype)  ### : np.ndarray (IN-space range)
        self.N      = N      ### number of defining points
        self.MO     = MO     ### int (dimension of OUT)
        self.kernel = kernel(self.IN.shape[0])
        self.seed   = seed ### define pseudo-random seed

        np.random.seed( self.seed )

        ### define anisotropic i.i.d white-noise
        if IN_noise is None:
            self.IN_noise=np.zeros(self.IN.shape[0], dtype=self.dtype)
        else:
            self.IN_noise = IN_noise
        if OUT_noise is None:
            self.OUT_noise=np.zeros(self.MO, dtype=self.dtype)
        else:
            self.OUT_noise = OUT_noise

        ### define IN-space defining-points
        self.R_ix  = np.random.uniform(0,1, (self.N, self.IN.shape[0])).astype(self.dtype)
        self.R_ix *= (self.IN[:,1] - self.IN[:,0])
        self.R_ix += self.IN[:,0]

        ### compute cholesky-factorization
        ### this will fail if K is not-PSD LinAlgError: Matrix is not positive definite
        try:
            L_ij = np.linalg.cholesky( self.kernel.K( self.R_ix ) ) ## not immutable
        except:
            #print("not PSD added to diag")
            L_ij = np.linalg.cholesky( self.kernel.K( self.R_ix ) + np.diag( 1.e-8 * np.random.rand(self.N).astype(self.dtype) ) )

        ### compute OUT-space defining-points
        D_iX  = np.random.normal(0,1,(self.N, self.MO)).astype(self.dtype)
        D_iX *= np.diag(L_ij)[:,None]
        D_iX  = np.matmul(L_ij, D_iX)

        self.S_jX  = scipy.linalg.cho_solve((L_ij, True), D_iX)

    def __call__(self, D_ax):
        """ evaluate for arbitrary values/points in OUT given points in IN.
        GIVEN >
              self
              D_ax : 2d-np.ndarray (D_ax ∈ IN)
        GET   >
              D_aX : 2d-np.ndarray (D_aX ∈ OUT, note captial 'X')
        """
        D_ax += self.IN_noise*np.random.normal(0,1,D_ax.shape).astype(self.dtype)
        D_aX  = np.matmul( self.kernel.K(D_ax, self.R_ix), self.S_jX )
        D_aX += self.OUT_noise*np.random.normal(0,1,D_aX.shape).astype(self.dtype)
        return D_aX

In [16]:
Domain = np.array([[0,10.],[-3,4.]], dtype=np.float64) #torch.tensor([[0,10.],[-3,4.],[-8,-2]]) ### numpy.2darray


f = RCF(Domain, 4, seed=48)

#### generate mesh to plot
R_ax = np.stack(np.meshgrid(*[ np.linspace(Domain[i,0], Domain[i,1], 20) for i in range(len(Domain)) ]), axis=-1)
#R_ax = R_ax.reshape((tf.prod( tf.asarray(R_ax.shape[:-1]) ), R_ax.shape[-1]))
shaper = np.concatenate((np.asarray(R_ax.shape[:-1], dtype=np.int64), R_ax.shape[-1]*np.ones(1, dtype=np.int64)))
R_ax = np.reshape(R_ax, (np.prod(R_ax.shape[:-1]), R_ax.shape[-1]))

R_ay = f(R_ax)

#### the plot
fig = go.Figure(data=[go.Scatter3d(x=R_ax[:,0], y=R_ax[:,1], z=R_ay[:,0], mode='markers'),
                      go.Scatter3d(x=f.R_ix[:,0], y=f.R_ix[:,1], z=f(f.R_ix)[:,0], mode='markers')])
fig.show()

## very small domain

In [19]:
Domain = np.array([[0,1.e-4],[-1.e-3,1.e-3]], dtype=np.float64) #torch.tensor([[0,10.],[-3,4.],[-8,-2]]) ### numpy.2darray


f = RCF(Domain, 41, seed=48)

#### generate mesh to plot
R_ax = np.stack(np.meshgrid(*[ np.linspace(Domain[i,0], Domain[i,1], 20) for i in range(len(Domain)) ]), axis=-1)
#R_ax = R_ax.reshape((tf.prod( tf.asarray(R_ax.shape[:-1]) ), R_ax.shape[-1]))
shaper = np.concatenate((np.asarray(R_ax.shape[:-1], dtype=np.int64), R_ax.shape[-1]*np.ones(1, dtype=np.int64)))
R_ax = np.reshape(R_ax, (np.prod(R_ax.shape[:-1]), R_ax.shape[-1]))

R_ay = f(R_ax)

#### the plot
fig = go.Figure(data=[go.Scatter3d(x=R_ax[:,0], y=R_ax[:,1], z=R_ay[:,0], mode='markers'),
                      go.Scatter3d(x=f.R_ix[:,0], y=f.R_ix[:,1], z=f(f.R_ix)[:,0], mode='markers')])
fig.show()

### how is this random function different from, just summing randomly distributed Gaussians (random means & random covariances)?

lets build this function...

### we need to detect when the matrix is not PSD, if the matrix is degenerate?? PSD can be degenerate, PD cannot be degenerate??

In [13]:
#i.i.d. gaussians....